In [26]:
%cd /home/tuyendv/Desktop/reranker
import json
import os
from glob import glob
from tqdm import tqdm
from underthesea import sent_tokenize
from utils import norm_text
import pandas as pd

/home/tuyendv/Desktop/reranker


In [22]:
def split_doc_in_sent_level(doc, max_length=256, max_overlap_length=128):
    chunks = []
    current_chunk = ""
    current_length = 0
    
    index = 0
    remain_overlap_length = max_overlap_length
    while index < len(doc):
        sent = doc[index]
        if current_length + len(sent.split(" ")) > max_length:
            chunks.append(current_chunk)
            while len(doc[index].split(" ")) < remain_overlap_length:
                remain_overlap_length -=len(doc[index].split(" "))
                if index == 0:
                    break
                index -= 1
            index += 1
            if index == len(doc):
                break
            current_chunk = doc[index]
            current_length = len(current_chunk.split(" "))
            remain_overlap_length = max_overlap_length
        else:
            current_chunk = current_chunk + " " + sent
            current_length = len(current_chunk.split(" "))
        
        index += 1
    chunks.append(current_chunk)
    
    return chunks

In [15]:
data_dir = "docs/raws/*.json"
docs = []
for _file in glob(data_dir):
    data = json.load(open(_file, "r", encoding="utf-8"))
    content = data["content"]
    
    docs.append(sent_tokenize(content))

In [23]:
chunks = []
for doc in docs:
    chunk = split_doc_in_sent_level(doc)
    chunks += chunk

In [27]:
passages = pd.DataFrame(chunks, columns=["passage_text"])
passages.head()

,passage_text
0,"""Đây là khoảnh khắc đặc biệt"", Haaland nói tạ..."
1,Tôi thực sự thích nó và có rất nhiều cầu thủ g...
2,Chúng tôi tập trung và đạt được điều mong muốn...
3,Tại giải cờ vua Đại kiện tướng quốc tế 2023 ở...
4,"12 GM trước đó lần lượt là Đào Thiên Hải, Từ H..."


In [28]:
passages.to_csv("/home/tuyendv/Desktop/reranker/docs/processed/passage.txt")

### train bm25

In [1]:
%cd /home/tuyendv/Desktop/reranker
import json
import os
from glob import glob
from tqdm import tqdm
from underthesea import sent_tokenize
from utils import norm_text
import pandas as pd
from bm25 import BM25

/home/tuyendv/Desktop/reranker


In [2]:
path="/home/tuyendv/Desktop/reranker/docs/processed/passage.txt"
passages = pd.read_csv(path, index_col=0)
passages.dropna(inplace=True)
passages.head()

,passage_text
0,"""Đây là khoảnh khắc đặc biệt"", Haaland nói tạ..."
1,Tôi thực sự thích nó và có rất nhiều cầu thủ g...
2,Chúng tôi tập trung và đạt được điều mong muốn...
3,Tại giải cờ vua Đại kiện tướng quốc tế 2023 ở...
4,"12 GM trước đó lần lượt là Đào Thiên Hải, Từ H..."


In [3]:
bm25_model = BM25()
bm25_model.train(passages)

In [4]:
path="bm25"
bm25_model.save(path)

In [6]:
query = "Haaland nói gì tại London"
bm25_model.search(query=query, topk=5)

[[11.48076,
  ' "Đây là khoảnh khắc đặc biệt", Haaland nói tại London tối 25/5, khi nhận giải Cầu thủ hay nhất Ngoại hạng Anh mùa này do Hiệp hội nhà báo bóng đá Anh (FWA) bầu chọn. "Gia nhập CLB đã ghi rất nhiều bàn và vô địch Ngoại hạng Anh nhiều lần, tôi kỳ vọng sẽ làm được những điều tốt đẹp, nhưng thực sự không ngờ mọi thứ lại tốt như vậy. Đây là mùa giải tuyệt vời và tôi rất vui".Haaland nhận giải Cầu thủ hay nhất Ngoại hạng Anh mùa 2022-2023 của FWA tối 25/5. Ảnh: mancity.comTiền đạo Na Uy cho biết có nhiều bạn và tận hưởng cuộc sống tại Manchester cả trong và ngoài sân cỏ. "Chúng tôi đùa rất nhiều, tận hưởng điều đó trong phòng thay đồ và trên sân. Tôi thực sự thích nó và có rất nhiều cầu thủ giỏi trong đội", Haaland nói thêm.Trong cuộc bầu chọn với hơn 800 thành viên của FWA, Haaland chiến thắng áp đảo với hơn 80% tổng số phiếu bầu - kỷ lục trong lịch sử Ngoại hạng Anh. Xếp sau lần lượt là Bukayo Saka, Martin Odegaard (Arsenal), Kevin de Bruyne (Man City) và Marcus Rashford (M